In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import relu
import os


In [ ]:
# Directories for training data
train_dir = '/content/drive/MyDrive/Training_Data'

# ImageDataGenerator for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% data for validation
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
)

# Training and validation data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Xception default size
    batch_size=32,
    class_mode='binary',
    subset='training',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation',
)



Found 2333 images belonging to 2 classes.
Found 583 images belonging to 2 classes.


In [ ]:
# Load the pre-trained Xception model without top layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation=relu)(x)  # Mish activation for experimentation
x = Dropout(0.5)(x)  # Dropout for regularization

# Output layer for binary classification (1 output unit with sigmoid)
predictions = Dense(1, activation='sigmoid')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define callbacks for early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('1024_relu_xception_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, checkpoint],
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 695s 8s/step - accuracy: 0.9282 - loss: 0.1671 - val_accuracy: 0.8851 - val_loss: 0.3745
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.9852 - loss: 0.0403 - val_accuracy: 0.9554 - val_loss: 0.1628
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 77s 978ms/step - accuracy: 0.9912 - loss: 0.0272 - val_accuracy: 0.9588 - val_loss: 0.1274
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 75s 964ms/step - accuracy: 0.9923 - loss: 0.0230 - val_accuracy: 0.9434 - val_loss: 0.2120
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.9946 - loss: 0.0180 - val_accuracy: 0.9571 - val_loss: 0.1250
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9911 - loss: 0.0280 - val_accuracy: 0.9468 - val_loss: 0.2188
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 76s 977ms/step - accuracy: 0.9882 - loss: 0.0292 - val_accuracy: 0.9760 - val_loss: 0.0923
Epoch 8/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9915 - loss: 0.0239 - val_accuracy: 0.9451 - val_loss

In [ ]:
# Define the ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Test data generator
test_dir = '/content/drive/MyDrive/Testing Data'  # Update with your test data directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),  # Same size used for training
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important: Do not shuffle test data to keep order
)



Found 705 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import load_model

# Load the best model saved during training
model = load_model('/content/1024_relu_xception_model.keras')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')




23/23 ━━━━━━━━━━━━━━━━━━━━ 153s 7s/step - accuracy: 0.9955 - loss: 0.0316
Test Loss: 0.039861589670181274
Test Accuracy: 0.9943262338638306


In [ ]:
# Load and preprocess an image for prediction
import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(img_path):
    # Load the image
    img = image.load_img(img_path, target_size=(299, 299))

    # Convert the image to array
    img_array = image.img_to_array(img)

    # Expand dimensions to match the shape of data in the model
    img_array = np.expand_dims(img_array, axis=0)

    # Rescale the image
    img_array /= 255.0

    return img_array

def predict_image(img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path)

    # Make a prediction
    prediction = model.predict(img_array)

    # Convert prediction to class label (assuming binary classification)
    if prediction[0] > 0.5:
        return "NoTumor"
    else:
        return "Glioma"

img_path = '/content/drive/MyDrive/Training_Data/glioma/Tr-glTr_0000.jpg'
result = predict_image(img_path)
print(f"The model predicts: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The model predicts: Glioma


In [ ]:
# Define the ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Test data generator
test_dir = '/content/drive/MyDrive/Testing Data'  # Update with your test data directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),  # Same size used for training
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Important: Do not shuffle test data to keep order
)



Found 705 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import load_model

# Load the best model saved during training
model = load_model('/content/128_relu_model.keras')




In [ ]:
# Evaluate the model
results = model.evaluate(test_generator)
test_loss, test_accuracy, test_auc = results  # Extract all three returned values

# Print the results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Test AUC: {test_auc}')

23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 288ms/step - AUC: 0.6182 - accuracy: 0.9938 - loss: 0.0646
Test Loss: 0.08934247493743896
Test Accuracy: 0.9914893507957458
Test AUC: 0.9937860369682312


In [ ]:
# Load and preprocess an image for prediction
import numpy as np
from tensorflow.keras.preprocessing import image

def preprocess_image(img_path):
    # Load the image
    img = image.load_img(img_path, target_size=(299, 299))

    # Convert the image to array
    img_array = image.img_to_array(img)

    # Expand dimensions to match the shape of data in the model
    img_array = np.expand_dims(img_array, axis=0)

    # Rescale the image
    img_array /= 255.0

    return img_array

def predict_image(img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path)

    # Make a prediction
    prediction = model.predict(img_array)

    # Convert prediction to class label (assuming binary classification)
    if prediction[0] > 0.5:
        return "NoTumor"
    else:
        return "Glioma"

img_path = '/content/drive/MyDrive/Training_Data/glioma/Tr-glTr_0000.jpg'
result = predict_image(img_path)
print(f"The model predicts: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The model predicts: Glioma


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Generate predictions on the test set
y_true = test_generator.classes  # True labels
y_pred_probs = model.predict(test_generator)  # Predicted probabilities
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to class labels

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate precision, recall, and F1 score
report = classification_report(y_true, y_pred, target_names=["Glioma", "NoTumor"])
print("Classification Report:")
print(report)


23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 285ms/step
Confusion Matrix:
[[300   0]
 [  6 399]]
Classification Report:
              precision    recall  f1-score   support

      Glioma       0.98      1.00      0.99       300
     NoTumor       1.00      0.99      0.99       405

    accuracy                           0.99       705
   macro avg       0.99      0.99      0.99       705
weighted avg       0.99      0.99      0.99       705



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import time  # For tracking time

# Custom Mish activation function


# Load base model (Xception) without the top classification layer
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze all layers except for the last 20 layers for fine-tuning
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Build the model with multiple hidden layers and Mish activation
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add multiple Dense layers with Mish activation
x = Dense(512, activation=mish)(x)  # First hidden layer with Mish
x = Dropout(0.5)(x)  # Dropout for regularization

x = Dense(256, activation=mish)(x)  # Second hidden layer with Mish
x = Dropout(0.5)(x)

x = Dense(128, activation=mish)(x)  # Third hidden layer with Mish
x = Dropout(0.5)(x)

# Output layer (1 unit with sigmoid for binary classification)
predictions = Dense(1, activation='sigmoid')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)



Found 2333 images belonging to 2 classes.
Found 583 images belonging to 2 classes.
Epoch 1/20


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import time  # For tracking time

In [ ]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [ ]:


# Directories for training data
train_dir = '/content/drive/MyDrive/Training_Data'

# ImageDataGenerator for training with augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% data for validation
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
)

# Training and validation data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Default size for Xception
    batch_size=32,
    class_mode='binary',
    subset='training',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation',
)

Found 2333 images belonging to 2 classes.
Found 583 images belonging to 2 classes.


In [ ]:
# Load base model (Xception) without the top classification layer
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze all layers except for the last 20 layers for fine-tuning
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Build the model with multiple hidden layers and Mish activation
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add multiple Dense layers with Mish activation
x = Dense(512, activation=mish)(x)  # First hidden layer with Mish
x = Dropout(0.5)(x)  # Dropout for regularization

x = Dense(256, activation=mish)(x)  # Second hidden layer with Mish
x = Dropout(0.5)(x)

x = Dense(128, activation=mish)(x)  # Third hidden layer with Mish
x = Dropout(0.5)(x)

# Output layer (1 unit with sigmoid for binary classification)
predictions = Dense(1, activation='sigmoid')(x)

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Compile the model with Adam optimizer and additional metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('mish_model.keras', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Track time taken for training
start_time = time.time()

# Train the model (without workers and use_multiprocessing)
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

# Calculate total time taken for training
end_time = time.time()
total_training_time = end_time - start_time
print(f"Total training time: {total_training_time} seconds")

# Test Data Generator for Evaluation
test_datagen = ImageDataGenerator(rescale=1./255)

test_dir = '/content/drive/MyDrive/Testing Data'  # Update with your test data directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),  # Same size used for training
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Do not shuffle test data
)

# Evaluate the model on test data
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 1948s 26s/step - AUC: 0.9676 - accuracy: 0.9235 - loss: 0.1727 - val_AUC: 0.8448 - val_accuracy: 0.7856 - val_loss: 3.0375 - learning_rate: 0.0010
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - AUC: 0.9993 - accuracy: 0.9951 - loss: 0.0168 